# Notebook per addestrare il classificatore

### Contesto
 Il dataset consiste in dati di collocamento nel mondo del lavoro da parte di studenti in un campus.
 Include dati sull'istruzione superiore, come specializzazione e punteggi. 
 Include anche dati sull'esperienza lavorativa e sulle offerte attese al primo impiego.

#### Obiettivo
 Addestrare un classificatore che, dato un candidato, preveda il piazzamento o meno. 
 In altre parole, individuate i fattori che contribuiscono al collocamento nel mondo del lavoro.

In [1871]:
# Import delle librerie e dei moduli utilizzati per l'addestramento

# Per la gestione dei dati
import numpy as np
import pandas as pd

# Disabilito questa opzione di pandas per evitare un warning bloccante ma non necessario durante il label encoding
pd.options.mode.chained_assignment = None  # default='warn'

# Importo Sklearn per gli strumenti necessari all'addestramento:

# Per pulire le variabili categoriche
from sklearn.preprocessing import LabelEncoder 

# Import del modello da addestrare
from sklearn.linear_model import LogisticRegression

# Per la valutazione del modello 
from sklearn.metrics import accuracy_score #

### Dataset

Il dataset contiene le caratteristiche di 173 studenti. 
La più importante è 'status', che assume solo valori 'Placed'/'Not Placed', ossia se lo studente abbia trovato collocazione nel suo primo anno.

- status - Collocato sì o no.
- gender - Maschio o Femmina
- ssc_p - Punteggio di scuola media
- ssc_b - Board of Education per le medie
- hsc_p - Punteggio di diploma
- hsc_b - Board of Education per il diploma
- hsc_s - Tipologia di liceo
- degree_p - Punteggio di laurea
- degree_t - Settore disciplinare della laurea
- workex - Esperienza lavorativa
- etest_p - Punteggio test di collocabilità (condotto dal college)
- specialisation - Specializzazione post laurea (MBA)
- mba_p - Punteggio MBA
- salary - Stipendio offerto dalla compagnia ai candidati.

## Data Loading

In [1872]:
# Carico i dati dal csv relativo ai dati di training creato durante la fase di preparazione
training_dataset = pd.read_csv('train.csv')

In [1873]:
# Check del training set
training_dataset.head()

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,salary,status
0,M,77.67,Others,64.89,Others,Commerce,70.67,Comm&Mgmt,No,89.0,Mkt&Fin,60.39,300000.0,Placed
1,M,84.20,Central,69.40,Central,Science,65.00,Sci&Tech,Yes,80.0,Mkt&HR,52.72,255000.0,Placed
2,M,52.00,Others,55.00,Others,Commerce,56.30,Comm&Mgmt,No,59.0,Mkt&Fin,64.74,NaN,Not Placed
3,M,63.00,Central,60.00,Central,Commerce,57.00,Comm&Mgmt,Yes,78.0,Mkt&Fin,54.55,204000.0,Placed
4,F,60.23,Central,69.00,Central,Science,66.00,Comm&Mgmt,No,72.0,Mkt&Fin,59.47,230000.0,Placed


In [1874]:
# Carico i dati dal csv relativo ai dati di test creato durante la fase di preparazione
test_dataset = pd.read_csv('test.csv')

In [1875]:
# Check del training set
test_dataset.head()

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,salary,status
0,M,52.0,Others,50.0,Others,Arts,61.0,Comm&Mgmt,No,60.0,Mkt&Fin,58.52,NaN,Not Placed
1,M,78.5,Central,65.5,Central,Science,67.0,Sci&Tech,Yes,95.0,Mkt&Fin,64.86,280000.0,Placed
2,M,62.0,Central,72.0,Central,Commerce,65.0,Comm&Mgmt,No,67.0,Mkt&Fin,56.49,216000.0,Placed
3,M,51.0,Central,44.0,Central,Commerce,57.0,Comm&Mgmt,No,64.0,Mkt&Fin,51.45,NaN,Not Placed
4,F,55.0,Central,67.0,Central,Commerce,64.0,Comm&Mgmt,No,60.0,Mkt&Fin,67.28,NaN,Not Placed


## Data Cleaning

In [1876]:
# Elimino la feature 'Salary' da entrambi i dataset in quanto è un dato 'a posteriori': rappresenta le offerte fatte 
# dalle aziende solo per studenti che sono già "placed". Non è utile quindi per determinare se uno studente sarà effettivamente
# assunto o meno
del[training_dataset['salary']]
del[test_dataset['salary']]

In [1877]:
# Label Encoding - trasformo le features categoriche in numeri
training_category_columns = training_dataset.select_dtypes(include = 'object') 

In [1878]:
training_number_columns = training_dataset.select_dtypes(include = np.number)  

In [1879]:
# Utilizzo il label encoder per potartare le feature categoriche a numeri
encoder = LabelEncoder()

for i in training_category_columns:
    training_category_columns[i] = encoder.fit_transform(training_category_columns[i])

In [1880]:
# Stessa operazione per i dati di test
test_category_columns = test_dataset.select_dtypes(include = 'object') 

In [1881]:
test_number_columns = test_dataset.select_dtypes(include = np.number)

In [1882]:
for i in test_category_columns:
    test_category_columns[i] = encoder.fit_transform(test_category_columns[i])

In [1883]:
# Ricompongo i dataset con le feature pulite
final_train = pd.concat([training_number_columns, training_category_columns], axis = 1)
final_test = pd.concat([test_number_columns, test_category_columns], axis = 1)

In [1884]:
# Definisco quali features sono del vettore d'input X e quale è la feature target Y
Y_train, Y_test = final_train['status'], final_test['status']

In [1885]:
# Definisco le feature del vettore d'input X, su cui effettuare l'addestramento e le predizioni
# A tal scopo è fondamentale eliminare la colonna relativa alla feature da predire!
X_train, X_test = final_train.drop(columns=['status']), final_test.drop(columns=['status'])

In [1886]:
# Check rapido dei vari dataset
Y_train.head()

0    1
1    1
2    0
3    1
4    1
Name: status, dtype: int32

In [1887]:
Y_test.head()

0    0
1    1
2    1
3    0
4    0
Name: status, dtype: int32

In [1888]:
X_train.head()

,ssc_p,hsc_p,degree_p,etest_p,mba_p,gender,ssc_b,hsc_b,hsc_s,degree_t,workex,specialisation
0,77.67,64.89,70.67,89.0,60.39,1,1,1,1,0,0,0
1,84.20,69.40,65.00,80.0,52.72,1,0,0,2,2,1,1
2,52.00,55.00,56.30,59.0,64.74,1,1,1,1,0,0,0
3,63.00,60.00,57.00,78.0,54.55,1,0,0,1,0,1,0
4,60.23,69.00,66.00,72.0,59.47,0,0,0,2,0,0,0


In [1889]:
X_test.head()

,ssc_p,hsc_p,degree_p,etest_p,mba_p,gender,ssc_b,hsc_b,hsc_s,degree_t,workex,specialisation
0,52.0,50.0,61.0,60.0,58.52,1,1,1,0,0,0,0
1,78.5,65.5,67.0,95.0,64.86,1,0,0,2,2,1,0
2,62.0,72.0,65.0,67.0,56.49,1,0,0,1,0,0,0
3,51.0,44.0,57.0,64.0,51.45,1,0,0,1,0,0,0
4,55.0,67.0,64.0,60.0,67.28,0,0,0,1,0,0,0


## Model Training

In [1890]:
# Ho scelto di utilizzare un LogistiRegressor data la presenza di feature continue nel dataset (le varie votazioni).
# Infine, ho scelto il solver liblinear in quanto è quello indicato per piccoli dataset, come questo.
model = LogisticRegression(solver='liblinear')

In [1891]:
# Addestro il modello con i dati di training
model.fit(X_train, Y_train)

LogisticRegression(solver='liblinear')

In [1892]:
# Utilizzo il modello addestrato per effettuare delle predizioni sui dati di train 
predictions_on_training = model.predict(X_train)

In [1893]:
# Controllo l'accuratezza sui dati di training 
print("Accuracy : "+str(accuracy_score(Y_train, predictions_on_training)))

Accuracy : 0.8759124087591241


In [1894]:
# Utilizzo il modello addestrato per effettuare predizioni sui dati di test 
predictions_on_test = model.predict(X_test)

## Model Assessment

In [1895]:
# Importo i moduli necessari per la valutazione del classificatore
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [1896]:
# Stampo l'accuratezza del classificatore sui dati di test
print("Accuracy : " + str(accuracy_score(Y_test, predictions_on_test)))

Accuracy : 0.9142857142857143


In [1897]:
# Stampo la matrice di confusione, per verificare l'accuratezza circa i valori predetti
print(confusion_matrix(Y_test, predictions_on_test))

[[10  3]
 [ 0 22]]


In [1898]:
# Stampo le principali metriche di classificazione
print(classification_report(Y_test, predictions_on_test))

              precision    recall  f1-score   support

           0       1.00      0.77      0.87        13
           1       0.88      1.00      0.94        22

    accuracy                           0.91        35
   macro avg       0.94      0.88      0.90        35
weighted avg       0.92      0.91      0.91        35

